In [19]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T


In [20]:
class NumpyLensDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        root_dir: path to 'train' or 'val' folder
        transform: optional callable for data augmentation or normalization
        """
        self.root_dir = root_dir
        self.transform = transform
        
        # Map folder names to numeric labels
        self.class_to_idx = {
            'no': 0,
            'sphere': 1,
            'vort': 2
        }
        
        # Gather all (file_path, label) pairs
        self.samples = []
        for class_name, label in self.class_to_idx.items():
            class_path = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_path):
                continue
            for fname in os.listdir(class_path):
                if fname.endswith('.npy'):
                    fpath = os.path.join(class_path, fname)
                    self.samples.append((fpath, label))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        npy_path, label = self.samples[idx]
        
        # Load the .npy file
        arr = np.load(npy_path)  # shape: (H, W) or (C, H, W)
        
        # If it's 2D, add a channel dimension: (1, H, W)
        if arr.ndim == 2:
            arr = arr[None, ...]
        
        # Convert to torch tensor
        tensor_data = torch.tensor(arr, dtype=torch.float32)
        
        # Apply transform (e.g., resize to 128x128)
        if self.transform:
            tensor_data = self.transform(tensor_data)
        
        return tensor_data, label


In [21]:
import torchvision.transforms.functional as TF

class ResizeTensor:
    def __init__(self, size=(128,128)):
        self.size = size
        
    def __call__(self, x):
        # x is a torch Tensor of shape [C, H, W]
        # We'll use interpolate for single-channel
        x = x.unsqueeze(0)  # shape [1, C, H, W] for interpolate
        x = F.interpolate(x, size=self.size, mode='bilinear', align_corners=False)
        x = x.squeeze(0)    # back to [C, H, W]
        return x

# Optionally, you can chain multiple transforms with T.Compose
transform = T.Compose([
    ResizeTensor((128,128)),  # Force 128x128
    # ... Add more if you like (normalization, etc.)
])


In [22]:
train_dir = '/Users/EndUser/Downloads/dataset/train'
val_dir   = '/Users/EndUser/Downloads/dataset/val'

train_dataset = NumpyLensDataset(root_dir=train_dir, transform=transform)
val_dataset   = NumpyLensDataset(root_dir=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [23]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        
        # We assume single-channel input => in_channels=1
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # After two pools, 128->64->32 => feature map is [16, 32, 32]
        self.fc1 = nn.Linear(16 * 32 * 32, 64)
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        # x shape: [B, 1, 128, 128]
        
        x = F.relu(self.conv1(x))   # [B, 8, 128, 128]
        x = self.pool(x)            # [B, 8, 64, 64]
        
        x = F.relu(self.conv2(x))   # [B, 16, 64, 64]
        x = self.pool(x)            # [B, 16, 32, 32]
        
        # Flatten
        x = x.view(x.size(0), -1)   # [B, 16*32*32] = [B, 16384]
        
        x = F.relu(self.fc1(x))     # [B, 64]
        x = self.fc2(x)             # [B, num_classes]
        return x


In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN(num_classes=3).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * data.size(0)
        _, preds = torch.max(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    train_loss = running_loss / total
    train_acc = correct / total
    
    # ---- VALIDATE ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            
            outputs = model(data)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * data.size(0)
            _, preds = torch.max(outputs, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    
    val_loss = val_loss / val_total
    val_acc = val_correct / val_total
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


Epoch [1/10] Train Loss: 1.0990, Train Acc: 0.3363 | Val Loss: 1.0988, Val Acc: 0.3333
Epoch [2/10] Train Loss: 1.0989, Train Acc: 0.3295 | Val Loss: 1.0988, Val Acc: 0.3333
Epoch [3/10] Train Loss: 1.0989, Train Acc: 0.3321 | Val Loss: 1.0986, Val Acc: 0.3333
Epoch [4/10] Train Loss: 1.0988, Train Acc: 0.3330 | Val Loss: 1.0987, Val Acc: 0.3333
Epoch [5/10] Train Loss: 1.0988, Train Acc: 0.3321 | Val Loss: 1.0988, Val Acc: 0.3333
Epoch [6/10] Train Loss: 1.0988, Train Acc: 0.3331 | Val Loss: 1.0987, Val Acc: 0.3333
Epoch [7/10] Train Loss: 1.0988, Train Acc: 0.3340 | Val Loss: 1.0987, Val Acc: 0.3333
Epoch [8/10] Train Loss: 1.0988, Train Acc: 0.3292 | Val Loss: 1.0988, Val Acc: 0.3333
Epoch [9/10] Train Loss: 1.0988, Train Acc: 0.3311 | Val Loss: 1.0988, Val Acc: 0.3333
Epoch [10/10] Train Loss: 1.0988, Train Acc: 0.3286 | Val Loss: 1.0986, Val Acc: 0.3333
